In [ ]:
import time, os, torch
import numpy as np
import open3d as o3d

torch.cuda.set_device(0)
root_path = os.getcwd()
os.chdir(root_path)
data_path = os.path.join(root_path, 'Data')
mat_path = os.path.join(data_path,'deform_mat')

import importlib
import deform_loader_test_train
importlib.reload(deform_loader_test_train)

In [ ]:
import sys
# sys.path = ['/home/young/Research/CoRL2021/Implicit_Deformable/', 
#     '/home/young/Research/CoRL2021/Implicit_Deformable/torchmeta/',
#     '/home/young/Research/CoRL2021/Implicit_Deformable',
#  '/home/young/Research/CoRL2021/Implicit_Deformable/torchmeta/modules/',
#  '/home/young/Research/CoRL2021/Implicit_Deformable/torchmeta/',
#  '/home/young/miniconda3/envs/py38/lib/python38.zip',
#  '/home/young/miniconda3/envs/py38/lib/python3.8',
#  '/home/young/miniconda3/envs/py38/lib/python3.8/lib-dynload',
#  '/home/young/miniconda3/envs/py38/lib/python3.8/site-packages',
#  '/home/young/miniconda3/envs/py38/lib/python3.8/site-packages/IPython/extensions',
#  '/home/young/.ipython']


import random
os.chdir(root_path)
import meta_modules, modules, loss_functions, training_hyper, deform_loader
import deform_loader_test_train
import deform_loader_sdf
from torch.utils.data import DataLoader
from importlib import reload
reload(deform_loader)
reload(meta_modules)

reload(training_hyper)
reload(loss_functions)
reload(deform_loader_sdf)
reload(deform_loader_test_train)

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

## Directories
experiment_name = 'After_corl_pretrain_no_grad_no_shared'
model_directory = os.path.join(root_path, 'logs', experiment_name)
sys.path.append(root_path)


# Nominal Shape Pre-train

## Integrate Data
### Format
     { 'Shape ID' : 
         {
         'nominal' : {'coords' : 1xNx3 tensor, 'gt' : 1xNx3 tensor, 'normals' : 1xNx3 tensor }, 
         'Deform ID' : {'coords' : 1xNx3 tensor, 'contact': 1xNx3 tensor, 'reaction': 1x3 tensor, 'gt' : 1xNx3 tensor, 'normals' : 1xNx3 tensor}
         } 
     }

# Main Training

In [ ]:
import pickle
os.chdir(data_path)
# with open('dataset_simul.pickle', 'rb') as pickle_file:
#     data_dict = pickle.load(pickle_file)
    
with open('virdo_simul_dataset.pickle', 'rb') as f:
    data_dict = pickle.load( f)
data_dict['train'][0][0]['scale']


In [ ]:
### Pair Shape code and Force code
import open3d as o3d
pairs = {}
data = data_dict['train']
for shape_idx in data: 
    pairs[shape_idx] = []
    for deform_idx in data[shape_idx]:
#         data[shape_idx][deform_idx]['scale'] =scale[shape_idx]
        if deform_idx != 'nominal':
            pcd_name = f'contact_{shape_idx}_{deform_idx}.ply'
            contact_pcd = o3d.geometry.PointCloud()
            contact_pcd_ = np.array(data[shape_idx][deform_idx]['contact']).reshape(-1,3)
            contact_pcd.points = o3d.utility.Vector3dVector(contact_pcd_ )
            o3d.io.write_point_cloud(pcd_name, contact_pcd)
            pairs[shape_idx].append(deform_idx)
print(pairs)

In [ ]:
import sys, os, json, torch
import numpy as np

import random
np.random.seed(0)
random.seed(0)

# experiment_name = '6obj' # Experiment folder you want to reconstruct
os.chdir(root_path)
model_directory = os.path.join(root_path, 'logs', experiment_name)


# General Parameters

lr =3e-4
num_epochs = 30000
epochs_til_ckpt = 400
steps_til_summary = 400
torch.manual_seed(500)
embedding_size = 16 #2 * num_shape
device  = torch.device('cuda') # ('cpu')#.to(A)
on_surface_points = 7000
embedding_size = 16#256
embedding_scale = 1
DEVICE = 'cuda'


'''
Train The Entire Model
'''
import torch.nn as nn
import torch, math
from torch.utils.tensorboard import SummaryWriter
from tqdm.autonotebook import tqdm
import shutil
import modules
import loss_functions, modules
from torch import nn
import utils
import importlib

In [ ]:


importlib.reload(modules)
importlib.reload(meta_modules)



## Shape Code
data = data_dict['train']
num_shape = len(data)
shape_latent_size = num_shape * 2
shape_embedding = torch.nn.Embedding(num_shape, shape_latent_size).requires_grad_(True).to(DEVICE)
torch.nn.init.normal_(shape_embedding.weight, mean=0.0, std=0.1)



## Define Shape Code
shape_model = meta_modules.hypo_deformation(in_features= 3, out_features= 1,
                                latent_dim = shape_latent_size, input_encoder = None)
shape_model.to(DEVICE)


In [ ]:
importlib.reload(loss_functions)             


## Directories
summaries_dir = os.path.join(model_directory, 'summaries_main_minimize_deform_rep')
utils.cond_mkdir(summaries_dir)
checkpoints_dir = os.path.join(model_directory, 'checkpoints_main_minimize_deform_rep')
utils.cond_mkdir(checkpoints_dir)
writer = SummaryWriter(summaries_dir)

# Define the loss
loss_sdf = loss_functions.hyper_loss
loss_hypo_weight = loss_functions.hypo_weight_loss
loss_emb =  loss_functions.latent_loss

# Pretraining

In [ ]:
# from chamferdist import ChamferDistance
from pytorch3d.loss import chamfer_distance
import matplotlib.pyplot as plt
from tqdm import tqdm
epochs= 40000

## Training
total_steps = 0
shape_model.eval()

optims = torch.optim.Adam([
            {'params': shape_embedding.parameters(), 'lr': 1e-4},
            {'params': shape_model.parameters(), 'lr': 1e-4}
          ])


with tqdm(total= len(data)* epochs) as pbar:
    for epoch in range(epochs):
        # Training
        tot_loss = 0
        feats = {'f_emb': {}, 'cnt_ft' : {}}
   
        for shape_idx in range ( len(data) ): 
            data_nom = data[shape_idx]['nominal']
            

            # Train nominal Shape
            shape_input_nom = {'coords': data_nom['coords'].to(DEVICE),
            'embedding' : shape_embedding(torch.tensor(shape_idx).to(DEVICE)).unsqueeze(0)}

            #orch.Size([1, 23462, 1]) torch.Size([1, 21200, 1])
            shape_nom_output = shape_model(shape_input_nom)
            nom_sdf_loss = loss_sdf(model_output=shape_nom_output, 
                             gt_sdf= data_nom['gt'].to(DEVICE),
                             gt_normals= data_nom['normals'].to(DEVICE), 
                            ks=1e6, ki=3e4, kn=5e1, kg=5e1)

            nom_sdf_losses = 0
            for loss_name, loss in  nom_sdf_loss.items():
                single_loss = loss.mean()
                writer.add_scalar(loss_name, single_loss, total_steps)
                nom_sdf_losses += single_loss


            hypo_losses = loss_hypo_weight(shape_nom_output)
            emb_losses = loss_emb(shape_embedding(torch.tensor(shape_idx).to(DEVICE)))


#             train_loss = deform_sdf_losses + nom_sdf_losses + 1e4*cd_loss + 1e2*hypo_losses + 1e2*emb_losses
            train_loss = nom_sdf_losses  + 1e2*hypo_losses + 1e2*emb_losses

            ## Update
            optims.zero_grad()
            train_loss.backward()
            optims.step()

            tot_loss += train_loss.detach() # YS
            pbar.update(1)
            writer.add_scalar("total_train_loss", train_loss, epoch)



        if not epoch % 10:
            tqdm.write(
                "Epoch %d, tot_loss %0.6f, train loss %0.6f" % (
                    epoch, tot_loss,  train_loss))
        
        if epoch % 2000 == 0 :
            torch.save({
                'epoch': num_epochs,
                'shape_model': shape_model.state_dict(),
                'shape_embedding': shape_embedding.weight.data},
                os.path.join(checkpoints_dir, f'shape_{epoch}.pth'))




In [ ]:
import utils
checkpoints_dir = os.path.join(model_directory, 'checkpoints_final')
utils.cond_mkdir(checkpoints_dir)

torch.save({
    'epoch': num_epochs,
    'shape_model': shape_model.state_dict(),
    'shape_embedding': shape_embedding.weight.data},
    os.path.join(checkpoints_dir, 'shape_final.pth'))


# Load Pretrained Model

In [ ]:
checkpoints_dir

In [ ]:
# checkpoints_dir = os.path.join(model_directory, 'checkpoints_final')
shape_latent_size = 12
checkpoints_dir = os.path.join(model_directory, 'checkpoints_main')
shape_path = os.path.join(checkpoints_dir,'shape_30000.pth')

shape_model_weight = torch.load(shape_path,map_location=torch.device('cuda'))
shape_embedding_weight = shape_model_weight['shape_embedding']
shape_model_weight = shape_model_weight['shape_model']
shape_embedding = torch.nn.Embedding.from_pretrained(shape_embedding_weight.to(device),freeze = True).to(device)


shape_model = meta_modules.hypo_deformation(in_features= 3, out_features= 1,
                                latent_dim = shape_latent_size, input_encoder = None)
shape_model.load_state_dict(shape_model_weight)
shape_model.to(DEVICE)
shape_model.eval()

## Marching Cube Reconstruction 

In [ ]:
import seaborn as sns; 
import sdf_meshing
import numpy as np
import matplotlib.pyplot as plt


class shape_Decoder(torch.nn.Module):
    def __init__(self,shape_model, shape_embedding):
        super().__init__()

        self.shape_embedding = shape_embedding.to(device) 
        self.shape_model = shape_model 

    def forward(self, coords):
        shape_input = {'coords': coords,'embedding' : self.shape_embedding}
        pred_sdf = self.shape_model(shape_input)
        return pred_sdf


for key, items in pairs.items():
    shape_emb = shape_embedding_weight[key]

    decoder = shape_Decoder(shape_model, shape_emb.unsqueeze(0))
    sdf_meshing.create_mesh(decoder, os.path.join(model_directory, f'nominal_{key}.xyz'), N= 400)
    torch.cuda.empty_cache()

In [ ]:
emb_s = shape_embedding.weight[5].unsqueeze(0)
resolution = 150

model = shape_Decoder(shape_model,emb_s)

qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out']

plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

plt.subplot(1,2,2)
ax = sns.heatmap(map)
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')

# Main Training

In [ ]:
importlib.reload(modules)
importlib.reload(meta_modules)


deform_latent_size = 32
deform_model = meta_modules.hypo_deformation(in_features= 3, out_features= 3,
                                             latent_dim = deform_latent_size + shape_latent_size, input_encoder = None)
deform_model.to(DEVICE)

classifier = modules.PointNetCls( k = deform_latent_size, feature_transform= False).to(DEVICE)


In [ ]:
model_directory

In [ ]:
importlib.reload(loss_functions)            
optims = torch.optim.Adam([
            {'params': shape_model.parameters(), 'lr': 1e-10},
              {'params': classifier.parameters(), 'lr': 1e-5},
              {'params': deform_model.parameters(), 'lr': 1e-5}
          ], lr=lr)



## Directories
summaries_dir = os.path.join(model_directory, 'summaries_main_minimize_deform_rep_higher_cdloss')
utils.cond_mkdir(summaries_dir)
checkpoints_dir = os.path.join(model_directory, 'checkpoints_main_minimize_deform_rephigher_cdloss')
utils.cond_mkdir(checkpoints_dir)
writer = SummaryWriter(summaries_dir)

# Define the loss
loss_sdf = loss_functions.hyper_loss
loss_hypo_weight = loss_functions.hypo_weight_loss
loss_emb =  loss_functions.latent_loss

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

def validation(epoch, shape_model, deform_model, shape_embedding_weight, deform_embedding_weight, device):
    class shape_Decoder(torch.nn.Module):
        def __init__(self,shape_model, deform_model, shape_embedding, deform_embedding, device):
            super().__init__()

            self.shape_embedding = shape_embedding.to(device) #.squeeze()
            self.deform_embedding = deform_embedding.to(device)
            self.shape_model = shape_model.to(device)
            self.deform_model = deform_model.to(device)

        def forward(self, coords):     
            
            
            deform_input={'coords':coords, 
                          'embedding':torch.cat([self.deform_embedding,self.shape_embedding],dim=-1)}
            deform_output = self.deform_model(deform_input)


            shape_input = {'coords': deform_output['model_in'].to(device),
                        'embedding' : self.shape_embedding}

            pred_sdf = self.shape_model(shape_input)
            return pred_sdf

    class deform_Decoder(torch.nn.Module):
        def __init__(self,shape_model, deform_model, shape_embedding, deform_embedding, device):
            super().__init__()

            self.shape_embedding = shape_embedding.to(device) #.squeeze()
            self.deform_embedding = deform_embedding.to(device)
            self.shape_model = shape_model.to(device)
            self.deform_model = deform_model.to(device)

        def forward(self, coords):

            deform_input={'coords':coords, 
                          'embedding':torch.cat([self.deform_embedding,self.shape_embedding],dim=-1)}
            deform_output = self.deform_model(deform_input)

            shape_input = {'coords': deform_output['model_in'].to(device),
                        'model_out' :deform_output['model_in'].to(device) + deform_output['model_out'],
                        'embedding' : self.shape_embedding}

            pred_sdf = self.shape_model(shape_input)
            return pred_sdf
        
    emb_s = shape_embedding_weight[5].unsqueeze(0)
    emb_d = deform_embedding_weight[120]
    resolution = 150
    model = shape_Decoder(shape_model, deform_model,emb_s,emb_d,device)
    qr = np.linspace(-1, 1, num=150)
    map = np.zeros((150,150))
    for iidx, i in enumerate(qr):
        for jidx, j in enumerate(qr):
            
            map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


    plt.figure(figsize=[5,5])

    ax = sns.heatmap(map,vmin=-0.01, vmax= 0.3)
    plt.contour(map, [0,0.09,0.19,0.29], colors='black')
    ax = sns.heatmap(map, cmap="Spectral", vmin=-0.01, vmax=0.3)


    plt.xlabel('X-axis')
    plt.ylabel('Z-axis')
    ax.collections[0].colorbar.set_label("Distance Value")  
    plt.show()

    for key, items in pairs.items():
        for itm in items:
 
            shape_emb = shape_embedding_weight[key]
            deform_emb = deform_embedding_weight[itm]

            decoder = deform_Decoder(shape_model,deform_model, shape_emb.unsqueeze(0), deform_emb, device)
            sdf_meshing.create_mesh(decoder, os.path.join(model_directory, f'{epoch}_{key}_{itm}.xyz'), N= 400)
            torch.cuda.empty_cache()
            break


In [ ]:
# from chamferdist import ChamferDistance

from pytorch3d.loss import chamfer_distance
from tqdm import tqdm
epochs=  15001
## Training
total_steps = 0
shape_model.eval()

with tqdm(total= len(data)* epochs) as pbar:
    for epoch in range(epochs):
        # Training
        tot_loss = 0
        feats = {'f_emb': {}, 'cnt_ft' : {}}
   
        for shape_idx in range ( len(data) ): ## YS    
            for deform_idx in data[shape_idx].keys():
                if deform_idx == "nominal":
                    continue
                    
                data_def = data[shape_idx][deform_idx]
                data_nom = data[shape_idx]['nominal']

                
                ## Forward
                reaction_force = data_def['reaction'].squeeze(0).float().to(DEVICE)
                contact_points = data_def['contact'].to(DEVICE)
                
                feat = classifier(contact_points.transpose(2, 1), reaction_force, epoch) # batchsize x points                       
                deform_input = {'coords': data_def['coords'].to(DEVICE),
                              'embedding':torch.cat([
                                  feat, shape_embedding(torch.tensor(shape_idx).to(DEVICE)).unsqueeze(0)
                              ], dim = -1)}

                
                deform_output = deform_model(deform_input)
                
                shape_input_def = {'coords':deform_output['model_in'],
                'model_out' : deform_output['model_in'] + deform_output['model_out'],
                'embedding' : shape_embedding(torch.tensor(shape_idx).to(DEVICE)).unsqueeze(0)}

                
                # Save Features 
                feats['cnt_ft'][deform_idx] = classifier.cnt_ft.detach().cpu()
                feats['f_emb'][deform_idx] = feat.detach().cpu()
                shape_def_output = shape_model(shape_input_def)
                
                
                deform_sdf_loss = loss_sdf(model_output= shape_def_output, 
                                       gt_sdf= data_def['gt'].to(DEVICE),
                                       gt_normals= data_def['normals'].to(DEVICE), 
                                       ks=1e6, ki=3e4, kn=5e1, kg=0)
                
                deform_sdf_losses = 0
                for loss_name, loss in deform_sdf_loss.items():
                    single_loss = loss.mean()
                    writer.add_scalar(loss_name, single_loss, total_steps)
                    deform_sdf_losses += single_loss

                    
                # Chamferdist between nominal and pushed back pointcloud
                def_on_surf_idx = torch.where(data_def['gt'] == 0)[1]  
                nom_on_surf_idx = torch.where(data_nom['gt'] == 0)[1]
                cd_loss  = chamfer_distance(shape_input_def['model_out'][:,def_on_surf_idx ,:],
                                                 data_nom['coords'][:,nom_on_surf_idx ,:].to(DEVICE))[0]
    
#                 cd_loss =  dist_bidirectional
#                 writer.add_scalar('CD', dist_bidirectional/ len(def_on_surf_idx) * 1000, total_steps)


                shape_input_nom = {'coords': data_nom['coords'].to(DEVICE),
                'embedding' : shape_embedding(torch.tensor(shape_idx).to(DEVICE)).unsqueeze(0)}
                 
                    
                shape_nom_output = shape_model(shape_input_nom)
                nom_sdf_loss = loss_sdf(model_output=shape_nom_output, 
                                 gt_sdf= data_nom['gt'].to(DEVICE),
                                 gt_normals= data_nom['normals'].to(DEVICE), 
                                ks=1e6, ki=3e4, kn=5e1, kg=0)
                
                nom_sdf_losses = 0
                for loss_name, loss in  nom_sdf_loss.items():
                    single_loss = loss.mean()
                    writer.add_scalar(loss_name, single_loss, total_steps)
                    nom_sdf_losses += single_loss


                hypo_losses = loss_hypo_weight(deform_output) + loss_hypo_weight(shape_def_output)
                emb_losses = loss_emb(feat) + loss_emb(shape_embedding(torch.tensor(shape_idx).to(DEVICE)))
                deform_loss = torch.norm(deform_output['model_out']) * 1
                
                
#                 train_loss = deform_sdf_losses + 5e4*cd_loss + 1e2*hypo_losses + 1e2*emb_losses
                train_loss = deform_loss + deform_sdf_losses + 5e4*cd_loss + 1e-3*nom_sdf_losses + 1e2*hypo_losses + 1e2*emb_losses
                
                
                ## Update
                optims.zero_grad()
                train_loss.backward()
                optims.step()

                tot_loss += train_loss.detach() # YS
                
                writer.add_scalar("hypo_losses", hypo_losses, total_steps)
                writer.add_scalar("emb_losses", emb_losses, total_steps)
                writer.add_scalar("total_train_loss", train_loss, epoch)               
            pbar.update(1)
        if not epoch % 10:
            tqdm.write(
                "Epoch %d, tot_loss %0.6f, train loss %0.6f" % (
                    epoch, tot_loss,  train_loss))

        total_steps += 1
#         if epoch % 500 == 0 and epoch != 0:
#             sch.step()
        if epoch % 1000 == 0 and epoch > 0:
            validation(epoch, shape_model, deform_model, shape_embedding.weight, feats['f_emb'], device)
            torch.save({
        'epoch': epoch,
        'shape_model': shape_model.state_dict(),
        'shape_embedding': shape_embedding.weight.data},
        os.path.join(checkpoints_dir, f'shape_{epoch}.pth'))
            torch.save({'epoch': epoch,
                        'deform_model': deform_model.state_dict()},
                        os.path.join(checkpoints_dir, f'deform_{epoch}.pth'))

            torch.save({'epoch': epoch,
                        'feats' : feats,
                        'force_model': classifier.state_dict(),
                       'optim': optims.state_dict()},
                        os.path.join(checkpoints_dir, f'force_final_{epoch}.pth'))




In [ ]:
# from chamferdist import ChamferDistance

from pytorch3d.loss import chamfer_distance
from tqdm import tqdm
epochs=  101
## Training
total_steps = 0
shape_model.eval()

with tqdm(total= len(data)* epochs) as pbar:
    for epoch in range(epochs):
        # Training
        tot_loss = 0
        feats = {'f_emb': {}, 'cnt_ft' : {}}
   
        for shape_idx in range ( len(data) ): ## YS    
            for deform_idx in data[shape_idx].keys():
                if deform_idx == "nominal":
                    continue
                    
                data_def = data[shape_idx][deform_idx]
                data_nom = data[shape_idx]['nominal']

                
                ## Forward
                reaction_force = data_def['reaction'].squeeze(0).float().to(DEVICE)
                contact_points = data_def['contact'].to(DEVICE)
                
                feat = classifier(contact_points.transpose(2, 1), reaction_force, epoch) # batchsize x points                       
                deform_input = {'coords': data_def['coords'].to(DEVICE),
                              'embedding':torch.cat([
                                  feat, shape_embedding(torch.tensor(shape_idx).to(DEVICE)).unsqueeze(0)
                              ], dim = -1)}

                
                deform_output = deform_model(deform_input)
                
                shape_input_def = {'coords':deform_output['model_in'],
                'model_out' : deform_output['model_in'] + deform_output['model_out'],
                'embedding' : shape_embedding(torch.tensor(shape_idx).to(DEVICE)).unsqueeze(0)}

                
                # Save Features 
                feats['cnt_ft'][deform_idx] = classifier.cnt_ft.detach().cpu()
                feats['f_emb'][deform_idx] = feat.detach().cpu()
                shape_def_output = shape_model(shape_input_def)
                
                
                deform_sdf_loss = loss_sdf(model_output= shape_def_output, 
                                       gt_sdf= data_def['gt'].to(DEVICE),
                                       gt_normals= data_def['normals'].to(DEVICE), 
                                       ks=1e6, ki=3e4, kn=5e1, kg=5e1)
                
                deform_sdf_losses = 0
                for loss_name, loss in deform_sdf_loss.items():
                    single_loss = loss.mean()
                    writer.add_scalar(loss_name, single_loss, total_steps)
                    deform_sdf_losses += single_loss

                    
                    
                    
                # Chamferdist between nominal and pushed back pointcloud
                def_on_surf_idx = torch.where(data_def['gt'] == 0)[1]  
                nom_on_surf_idx = torch.where(data_nom['gt'] == 0)[1]
                dist_bidirectional = chamfer_distance(shape_input_def['model_out'][:,def_on_surf_idx ,:],
                                                 data_nom['coords'][:,nom_on_surf_idx ,:].to(DEVICE))[0]
    
                cd_loss =  dist_bidirectional/ len(def_on_surf_idx)
                writer.add_scalar('CD', dist_bidirectional/ len(def_on_surf_idx) * 1000, total_steps)


                shape_input_nom = {'coords': data_nom['coords'].to(DEVICE),
                'embedding' : shape_embedding(torch.tensor(shape_idx).to(DEVICE)).unsqueeze(0)}
                 
                    
                shape_nom_output = shape_model(shape_input_nom)
                nom_sdf_loss = loss_sdf(model_output=shape_nom_output, 
                                 gt_sdf= data_nom['gt'].to(DEVICE),
                                 gt_normals= data_nom['normals'].to(DEVICE), 
                                ks=1e6, ki=3e4, kn=5e1, kg=5e1)
                
                nom_sdf_losses = 0
                for loss_name, loss in  nom_sdf_loss.items():
                    single_loss = loss.mean()
                    writer.add_scalar(loss_name, single_loss, total_steps)
                    nom_sdf_losses += single_loss


                hypo_losses = loss_hypo_weight(deform_output) + loss_hypo_weight(shape_def_output)
                emb_losses = loss_emb(feat) + loss_emb(shape_embedding(torch.tensor(shape_idx).to(DEVICE)))
                deform_loss = torch.norm(deform_output['model_out']).mean() * 1e-1
                
                
#                 train_loss = deform_sdf_losses + 5e4*cd_loss + 1e2*hypo_losses + 1e2*emb_losses
                train_loss = deform_loss + deform_sdf_losses + 5e4*cd_loss + 1e-3*nom_sdf_losses + 1e2*hypo_losses + 1e2*emb_losses
                
                
                ## Update
                optims.zero_grad()
                train_loss.backward()
                optims.step()

                tot_loss += train_loss.detach() # YS
                
                writer.add_scalar("hypo_losses", hypo_losses, total_steps)
                writer.add_scalar("emb_losses", emb_losses, total_steps)
                writer.add_scalar("total_train_loss", train_loss, epoch)               
            pbar.update(1)
        if not epoch % 10:
            tqdm.write(
                "Epoch %d, tot_loss %0.6f, train loss %0.6f" % (
                    epoch, tot_loss,  train_loss))

#         total_steps += 1
# #         if epoch % 500 == 0 and epoch != 0:
# #             sch.step()
#         if epoch % 1000 == 0 :
            validation(epoch, shape_model, deform_model, shape_embedding.weight, feats['f_emb'], device)
            torch.save({
        'epoch': epoch,
        'shape_model': shape_model.state_dict(),
        'shape_embedding': shape_embedding.weight.data},
        os.path.join(checkpoints_dir, f'shape_{epoch}.pth'))
            torch.save({'epoch': epoch,
                        'deform_model': deform_model.state_dict()},
                        os.path.join(checkpoints_dir, f'deform_{epoch}.pth'))

            torch.save({'epoch': epoch,
                        'feats' : feats,
                        'force_model': classifier.state_dict(),
                       'optim': optims.state_dict()},
                        os.path.join(checkpoints_dir, f'force_final_{epoch}.pth'))

In [ ]:
            validation(epoch, shape_model, deform_model, shape_embedding.weight, feats['f_emb'], device)
            torch.save({
        'epoch': epoch,
        'shape_model': shape_model.state_dict(),
        'shape_embedding': shape_embedding.weight.data},
        os.path.join(checkpoints_dir, f'shape_{epoch}.pth'))
            torch.save({'epoch': epoch,
                        'deform_model': deform_model.state_dict()},
                        os.path.join(checkpoints_dir, f'deform_{epoch}.pth'))

            torch.save({'epoch': epoch,
                        'feats' : feats,
                        'force_model': classifier.state_dict(),
                       'optim': optims.state_dict()},
                        os.path.join(checkpoints_dir, f'force_final_{epoch}.pth'))

### Continue Training

In [ ]:


torch.save({
        'epoch': epoch,
        'shape_model': shape_model.state_dict(),
        'shape_embedding': shape_embedding.weight.data},
        os.path.join(checkpoints_dir, f'shape_final.pth'))
torch.save({'epoch': epoch,
                        'deform_model': deform_model.state_dict()},
                        os.path.join(checkpoints_dir, f'deform_final.pth'))

torch.save({'epoch': epoch,
                        'feats' : feats,
                        'force_model': classifier.state_dict()},
                        os.path.join(checkpoints_dir, f'force_final.pth'))

In [ ]:
classifier_log['force_model']['force_mlp.fc5.weight'].shape

# Reconstruction

In [ ]:
checkpoints_dir

In [ ]:
 os.chdir(root_path)
import torch, os
import meta_modules

import importlib, os, sys, torch
import modules, utils
import sdf_meshing
import configargparse
import diff_operators
import meta_modules
importlib.reload(sdf_meshing)
importlib.reload(meta_modules)
importlib.reload(modules)



epoch = 10000
classifier_log = torch.load(os.path.join(checkpoints_dir, f'force_final_{epoch}.pth'),
                            map_location=torch.device('cpu'))
classifier_model_log = modules.PointNetCls(k = deform_latent_size, feature_transform= False).to(DEVICE)
classifier_model_log.load_state_dict(classifier_log['force_model'])

class shape_Decoder(torch.nn.Module):
    def __init__(self,shape_model, deform_model, shape_embedding, deform_embedding, device):
        super().__init__()

        self.shape_embedding = shape_embedding.to(device) #.squeeze()
        self.deform_embedding = deform_embedding.to(device)
        self.shape_model = shape_model.to(device)
        self.deform_model = deform_model.to(device)

    def forward(self, coords, type = 'deformed'):
  
        deform_input={'coords':coords, 
                      'embedding':torch.cat([self.deform_embedding,self.shape_embedding],dim=-1)}
        deform_output = self.deform_model(deform_input)

        shape_input = {'coords': deform_output['model_in'].to(device),
                    'model_out' :deform_output['model_in'].to(device) + deform_output['model_out'],
                    'embedding' : self.shape_embedding}
        
        shape_input = {'coords': deform_output['model_in'].to(device),
                    'embedding' : self.shape_embedding}
        
        pred_sdf = self.shape_model(shape_input)
        return pred_sdf

log_shape = torch.load(os.path.join(checkpoints_dir,f'shape_{epoch}.pth'),map_location=torch.device('cpu'))

shape_embedding_weight = log_shape['shape_embedding']
shape_embedding_log = torch.nn.Embedding.from_pretrained(shape_embedding_weight ).float().to(DEVICE)
shape_model_weight = log_shape['shape_model']

shape_latent_size = shape_embedding_weight.shape[1]
# shape_model_log = meta_modules.hypo_shape(in_features= 3, out_features= 1,
#                                         latent_dim = shape_latent_size, input_encoder = None)

shape_model_log = meta_modules.hypo_deformation(in_features= 3, out_features= 1,
                                        latent_dim = shape_latent_size, input_encoder = None)



## Shape module + Deformation Module/code
deform_summary_path = os.path.join(checkpoints_dir,f'deform_{epoch}.pth')
deform_summary = torch.load(deform_summary_path,map_location=torch.device('cpu'))
deform_model_weight = deform_summary['deform_model']
try:
    feats = classifier_log['feats']
    
except : 
    feats = deform_summary['deform_embedding']
deform_embedding_weight = feats['f_emb']
cnt_ft = feats['cnt_ft']

deform_latent_size = deform_embedding_weight[48].size()[1]
# deform_model_log = meta_modules.hypo_shape(in_features= 3, out_features= 3,
#                                            latent_dim = deform_latent_size + shape_latent_size , 
#                                        input_encoder = None)

deform_model_log = meta_modules.hypo_deformation(in_features= 3, out_features= 3,
                                           latent_dim = deform_latent_size + shape_latent_size , 
                                       input_encoder = None)

deform_model_log.load_state_dict(deform_model_weight)
deform_model_log.to(device)
deform_model_log.eval()

shape_model_log.load_state_dict(shape_model_weight)
shape_model_log.to(device)
shape_model_log.eval()

class shape_Decoder(torch.nn.Module):
    def __init__(self,shape_model, deform_model, shape_embedding, deform_embedding, device):
        super().__init__()

        self.shape_embedding = shape_embedding.to(device) #.squeeze()
        self.deform_embedding = deform_embedding.to(device)
        self.shape_model = shape_model.to(device)
        self.deform_model = deform_model.to(device)

    def forward(self, coords, type = 'deformed'):
  
        deform_input={'coords':coords, 
                      'embedding':torch.cat([self.deform_embedding,self.shape_embedding],dim=-1)}
        deform_output = self.deform_model(deform_input)

        shape_input = {'coords': deform_output['model_in'].to(device),
                    'model_out' :deform_output['model_in'].to(device) + deform_output['model_out'],
                    'embedding' : self.shape_embedding}
        
#         shape_input = {'coords': deform_output['model_in'].to(device),
#                     'embedding' : self.shape_embedding}
        
        pred_sdf = self.shape_model(shape_input)
        return pred_sdf

In [ ]:
## Reconstruct with Marching Cube

for key, items in pairs.items():
    for itm in items:
        shape_emb = shape_embedding_weight[key]
        deform_emb = deform_embedding_weight[itm]
        
        decoder = shape_Decoder(shape_model_log,deform_model_log, shape_emb.unsqueeze(0), deform_emb, device)
        sdf_meshing.create_mesh(decoder, os.path.join(model_directory, f'final_{key}_{itm}.xyz'), N= 400)
        torch.cuda.empty_cache()


In [ ]:
key = 1
itm = 3


itm1 = 25
itm2 = 42  
shape_emb = shape_embedding_weight[key]
# deform_emb = (deform_embedding_weight[itm1]+ deform_embedding_weight[itm2])/2

deform_emb = 1.25* deform_embedding_weight[itm1] - 0.25*deform_embedding_weight[itm2]

decoder = shape_Decoder(shape_model_log,deform_model_log, shape_emb.unsqueeze(0), deform_emb, device)
# sdf_meshing.create_mesh(decoder, os.path.join(model_directory, f'Best_ori_{key}_{itm}_.xyz'), N= 512)
sdf_meshing.create_mesh(decoder, os.path.join(model_directory, f'Experiment_{key}_{itm1}_{itm2}_1.xyz'), N= 512)
torch.cuda.empty_cache()


## Contact Location Decoder

In [ ]:
importlib.reload(modules)
cnt_decoder = modules.CntDecoder(shape_latent_size + deform_latent_size , 6)
cnt_decoder.to(DEVICE)
# nn.init.xavier_uniform_(cnt_decoder.parameters(), gain=nn.init.calculate_gain('relu'))

In [ ]:
cnt_sum_est_list = []
with tqdm(total=len(main_dataloader)* 1 ) as pbar:
    for model_input, gt_ori, idx in main_dataloader: ## YS
        mode = 'all'


        force_idx = gt_ori['deform_idx'].tolist()[0]
        object_idx = gt_ori['shape_idx'][0].to(DEVICE)

        deform_emb = torch.cat([deform_embedding_weight[force_idx].to(DEVICE), 
                                shape_embedding_log(object_idx).unsqueeze(0)], dim = 1)
        cnt_sum_est = cnt_decoder(deform_emb.to(DEVICE))                       
        cnt_sum_est_list.append(cnt_sum_est)
        
        error = model_input['contact_summary'].to(DEVICE) - cnt_sum_est # l1 loss
        train_loss = abs(error).mean()
        print(model_input['contact_summary'].to(DEVICE))

cnt_sum_est_list = torch.cat(cnt_sum_est_list, dim = 0)

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[5].unsqueeze(0)
emb_d = deform_embedding_weight[120]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d,device)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()

        
plt.figure(figsize=[5,5])

ax = sns.heatmap(map,vmin=-0.01, vmax= 0.3)
plt.contour(map, [0,0.09,0.19,0.29], colors='black')
ax = sns.heatmap(map, cmap="Spectral", vmin=-0.01, vmax=0.3)


plt.xlabel('X-axis')
plt.ylabel('Z-axis')        

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[5].unsqueeze(0)
emb_d = deform_embedding_weight[120]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d,device)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map,vmin=-0.01, vmax= 0.3)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

plt.subplot(1,2,2)
ax = sns.heatmap(map,vmin=-0.01, vmax= 0.3)
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')


In [ ]:
plt.figure(figsize=[5,5])

ax = sns.heatmap(map,vmin=-0.01, vmax= 0.3)
plt.contour(map, [0,0.09,0.19,0.29], colors='black')
ax = sns.heatmap(map, cmap="Spectral", vmin=-0.01, vmax=0.3)


plt.xlabel('X-axis')
plt.ylabel('Z-axis')

In [ ]:
plt.figure(figsize=[5,5])

ax = sns.heatmap(map,vmin=-0.01, vmax= 0.3)
plt.contour(map, [0,0.09,0.19,0.29], colors='black')
ax = sns.heatmap(map, cmap="Spectral", vmin=-0.01, vmax=0.3)


plt.xlabel('X-axis')
plt.ylabel('Z-axis')

In [ ]:

emb_s = shape_embedding_weight[5].unsqueeze(0)
emb_d = deform_embedding_weight[120]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d,device)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.figure(figsize=[5,5])

ax = sns.heatmap(map,vmin=-0.01, vmax= 0.3)
plt.contour(map, [0,0.09,0.19,0.29], colors='black')
ax = sns.heatmap(map, cmap="Spectral", vmin=-0.01, vmax=0.3)


plt.xlabel('X-axis')
plt.ylabel('Z-axis')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=[5,5])

ax = sns.heatmap(map,vmin=-0.01, vmax= 0.3)
plt.contour(map, [0,0.09,0.19,0.29], colors='black')
ax = sns.heatmap(map, cmap="Spectral", vmin=-0.01, vmax=0.3)


plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

In [ ]:
import seaborn as sns; 
class shape_Decoder(torch.nn.Module):
    def __init__(self,shape_model, deform_model, shape_embedding, deform_embedding, device):
        super().__init__()

        self.shape_embedding = shape_embedding.to(device) #.squeeze()
        self.deform_embedding = deform_embedding.to(device)
        self.shape_model = shape_model.to(device)
        self.deform_model = deform_model.to(device)

    def forward(self, coords, type = 'deformed'):
  
        deform_input={'coords':coords, 
                      'embedding':torch.cat([self.deform_embedding,self.shape_embedding],dim=-1)}
        deform_output = self.deform_model(deform_input)

        shape_input = {'coords': deform_output['model_in'].to(device),
                    'embedding' : self.shape_embedding}
        
        pred_sdf = self.shape_model(shape_input)
        return pred_sdf

emb_s = shape_embedding_weight[5].unsqueeze(0)
emb_d = deform_embedding_weight[120]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d,device)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.figure(figsize=[5,5])

ax = sns.heatmap(map,vmin=-0.01, vmax= 0.3)
plt.contour(map, [0,0.09,0.19,0.29], colors='black')
ax = sns.heatmap(map, cmap="Spectral", vmin=-0.01, vmax=0.3)


plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[2].unsqueeze(0)
emb_d = deform_embedding_weight[50]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d,device)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map,vmin=-0.01, vmax= 0.3)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

plt.subplot(1,2,2)
ax = sns.heatmap(map,vmin=-0.01, vmax= 0.3)
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')


In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[0].unsqueeze(0)
emb_d = deform_embedding_weight[5]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d,device)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

plt.subplot(1,2,2)
ax = sns.heatmap(map)
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[2].unsqueeze(0)
emb_d = deform_embedding_weight[50]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d,device)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

plt.subplot(1,2,2)
ax = sns.heatmap(map)
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[0].unsqueeze(0)
emb_d = deform_embedding_weight[5]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d,device)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

plt.subplot(1,2,2)
ax = sns.heatmap(map)
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[2].unsqueeze(0)
emb_d = deform_embedding_weight[2]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d,device)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

plt.subplot(1,2,2)
ax = sns.heatmap(map)
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[0].unsqueeze(0)
emb_d = deform_embedding_weight[0]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

# map = np.zeros((150,150))
# for iidx, i in enumerate(qr):
#     for jidx, j in enumerate(qr):
#         map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

# plt.subplot(1,2,2)
# ax = sns.heatmap(map)
# plt.xlabel('Y-axis')
# plt.ylabel('Z-axis')

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[0].unsqueeze(0)
emb_d = deform_embedding_weight[0]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

# map = np.zeros((150,150))
# for iidx, i in enumerate(qr):
#     for jidx, j in enumerate(qr):
#         map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

# plt.subplot(1,2,2)
# ax = sns.heatmap(map)
# plt.xlabel('Y-axis')
# plt.ylabel('Z-axis')

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[0].unsqueeze(0)
emb_d = deform_embedding_weight[0]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

# map = np.zeros((150,150))
# for iidx, i in enumerate(qr):
#     for jidx, j in enumerate(qr):
#         map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

# plt.subplot(1,2,2)
# ax = sns.heatmap(map)
# plt.xlabel('Y-axis')
# plt.ylabel('Z-axis')

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[0].unsqueeze(0)
emb_d = deform_embedding_weight[1]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

# map = np.zeros((150,150))
# for iidx, i in enumerate(qr):
#     for jidx, j in enumerate(qr):
#         map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

# plt.subplot(1,2,2)
# ax = sns.heatmap(map)
# plt.xlabel('Y-axis')
# plt.ylabel('Z-axis')

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[0].unsqueeze(0)
emb_d = deform_embedding_weight[1]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

# map = np.zeros((150,150))
# for iidx, i in enumerate(qr):
#     for jidx, j in enumerate(qr):
#         map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

# plt.subplot(1,2,2)
# ax = sns.heatmap(map)
# plt.xlabel('Y-axis')
# plt.ylabel('Z-axis')

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[3].unsqueeze(0)
emb_d = deform_embedding_weight[29]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out'].detach().cpu()


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

# map = np.zeros((150,150))
# for iidx, i in enumerate(qr):
#     for jidx, j in enumerate(qr):
#         map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

# plt.subplot(1,2,2)
# ax = sns.heatmap(map)
# plt.xlabel('Y-axis')
# plt.ylabel('Z-axis')
# ax.collections[0].colorbar.set_label("Distance Value")

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[3].unsqueeze(0)
emb_d = deform_embedding_weight[28]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out']


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

plt.subplot(1,2,2)
ax = sns.heatmap(map)
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[2].unsqueeze(0)
emb_d = deform_embedding_weight[39]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out']


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

plt.subplot(1,2,2)
ax = sns.heatmap(map)
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[3].unsqueeze(0)
emb_d = deform_embedding_weight[28]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out']


plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

plt.subplot(1,2,2)
ax = sns.heatmap(map)
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")


In [ ]:
import seaborn as sns; 
import numpy as np
import matplotlib.pyplot as plt

emb_s = shape_embedding_weight[2].unsqueeze(0)
emb_d = deform_embedding_weight[39]
resolution = 150
model = shape_Decoder(shape_model_log, deform_model_log,emb_s,emb_d)
qr = np.linspace(-1, 1, num=150)
map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[i,0,j]]).float().cuda())['model_out']

plt.figure(size=(10,5))
plt.subplot(1,2,1)
ax = sns.heatmap(map)
plt.xlabel('X-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")

map = np.zeros((150,150))
for iidx, i in enumerate(qr):
    for jidx, j in enumerate(qr):
        map[resolution-jidx-1,iidx] = model(torch.tensor([[0,i,j]]).float().cuda())['model_out']

plt.subplot(1,2,2)
ax = sns.heatmap(map)
plt.xlabel('Y-axis')
plt.ylabel('Z-axis')
ax.collections[0].colorbar.set_label("Distance Value")


# Experiments

In [ ]:
!pip install plotly
!pip install umap-learn

In [ ]:
df_deform_emb = {}
tot = []
label = []

for obj_idx, deform_idxs in pairs.items():
  for deform_idx in deform_idxs:
    tot.append(deform_embedding_weight[deform_idx])
    label.append(obj_idx)

tot = torch.cat(tot,dim=0)

import umap.umap_ as umap
reducer = umap.UMAP()
embedding = reducer.fit_transform(tot)
embedding.shape

import matplotlib.pyplot as plt
import seaborn as sns




In [ ]:

plt.figure(figsize=(4, 4))
plt.scatter(embedding[:, 0], embedding[:, 1], c=[sns.color_palette()[x] for x in label], s = 10)
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of force codes', fontsize=12);
plt.xlim(-5,15)
plt.ylim(-5,15)

# [0,1,2] = blue, orange, green


In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook()
# output_file("line.html")
tools = "hover, box_zoom, undo, crosshair"
p = figure(plot_width=400, plot_height=400, tools=tools)

p.circle(embedding[:, 0], embedding[:, 1], size=20, color=[sns.color_palette()[x] for x in label], alpha=0.5)

# show the results
show(p)

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook()
# output_file("line.html")
tools = "hover, box_zoom, undo, crosshair"
p = figure(plot_width=400, plot_height=400, tools=tools)

p.circle(embedding[:, 0], embedding[:, 1], size=20, color=[sns.color_palette()[x] for x in label], alpha=0.5)

# show the results
show(p)

### transfer to other objects


In [ ]:
key = 1


for itm in pairs[0]:
  shape_emb = shape_embedding_weight[key]
  deform_emb = deform_embedding_weight[itm]
  decoder = shape_Decoder(shape_model_log,deform_model_log, shape_emb.unsqueeze(0), deform_emb, device)
  sdf_meshing.create_mesh(decoder, os.path.join(model_directory, f'Deformed_{key}_{itm}_.xyz'), N= 400)
  torch.cuda.empty_cache()

## Table 1a. Nominal Shape CD and L1 

In [ ]:
## Undeformed Decoder
class Decoder(torch.nn.Module):
    def __init__(self,shape_model,shape_embedding,device):
        super().__init__()

        self.shape_embedding = shape_embedding.to(device) #.squeeze()
        self.shape_model = shape_model.to(device)
    def forward(self, coords):
        
        shape_input = {'coords':coords.unsqueeze(0),'embedding': self.shape_embedding}
        pred_sdf = self.shape_model(shape_input)

        return pred_sdf

In [ ]:
import logging
import numpy as np
import plyfile
import skimage.measure
import time
import torch
import diff_operators


def create_mesh(
    decoder, filename, N=256, max_batch=40 ** 3, offset=None, scale=None
):
    start = time.time()
    ply_filename = filename



    # NOTE: the voxel_origin is actually the (bottom, left, down) corner, not the middle
    voxel_origin = [-1, -1, -1]
    voxel_size = 2.0 / (N - 1)

    overall_index = torch.arange(0, N ** 3, 1, out=torch.LongTensor())
    samples = torch.zeros(N ** 3, 4)
    normals = torch.zeros(N **3, 3)

    # transform first 3 columns
    # to be the x, y, z index
    samples[:, 2] = overall_index % N
    samples[:, 1] = (overall_index.long() / N) % N
    samples[:, 0] = ((overall_index.long() / N) / N) % N

    # transform first 3 columns
    # to be the x, y, z coordinate
    samples[:, 0] = (samples[:, 0] * voxel_size) + voxel_origin[2]
    samples[:, 1] = (samples[:, 1] * voxel_size) + voxel_origin[1]
    samples[:, 2] = (samples[:, 2] * voxel_size) + voxel_origin[0]

    num_samples = N ** 3

    samples.requires_grad = False

    head = 0
    
    decoder.eval()
    while head < num_samples:
        sample_subset = samples[head : min(head + max_batch, num_samples), 0:3].cuda()
        output = decoder(sample_subset)

        
        samples[head : min(head + max_batch, num_samples), 3] = (
            output['model_out']
            .squeeze()#.squeeze(1)
            .detach()
            .cpu()
        )

        head += max_batch
        
        del output['model_out']
        del output['model_in']
        torch.cuda.empty_cache()
 

    sdf_values = samples[:, 3]
    sdf_values = sdf_values.reshape(N, N, N)

    end = time.time()
    print("sampling takes: %f" % (end - start))

    pc = convert_sdf_samples_to_ply(
        sdf_values.data.cpu(),
        voxel_origin,
        voxel_size,
        ply_filename[:-4] + ".ply",
        offset,
        scale,
    )
    return pc


def convert_sdf_samples_to_ply(
    pytorch_3d_sdf_tensor,
    voxel_grid_origin,
    voxel_size,
    ply_filename_out,
    offset=None,
    scale=None,
):
    """
    Convert sdf samples to .ply
    :param pytorch_3d_sdf_tensor: a torch.FloatTensor of shape (n,n,n)
    :voxel_grid_origin: a list of three floats: the bottom, left, down origin of the voxel grid
    :voxel_size: float, the size of the voxels
    :ply_filename_out: string, path of the filename to save to
    This function adapted from: https://github.com/RobotLocomotion/spartan
    """

    start_time = time.time()

    numpy_3d_sdf_tensor = pytorch_3d_sdf_tensor.numpy()

    verts, faces, normals, values = np.zeros((0, 3)), np.zeros((0, 3)), np.zeros((0, 3)), np.zeros(0)
    try:
        verts, faces, normals, values = skimage.measure.marching_cubes_lewiner(
            numpy_3d_sdf_tensor, level=0.0, spacing=[voxel_size] * 3
        )
    except:
        pass

    # transform from voxel coordinates to camera coordinates
    # note x and y are flipped in the output of marching_cubes
    mesh_points = np.zeros_like(verts)
    mesh_points[:, 0] = voxel_grid_origin[0] + verts[:, 0]
    mesh_points[:, 1] = voxel_grid_origin[1] + verts[:, 1]
    mesh_points[:, 2] = voxel_grid_origin[2] + verts[:, 2]

    return mesh_points

In [ ]:
from pytorch3d.loss import chamfer_distance
import matplotlib.pyplot as plt
np.random.seed(0)
random.seed(0)

dist_lst = []
dist_lst_real = []
n = 5600
data = data_dict["train"]
for shape_idx in range ( len(data) ): ## YS    
    for deform_idx in data[shape_idx].keys():
        if deform_idx != "nominal":
            continue
            
        data_i = data[shape_idx][deform_idx]
        shape_emb = shape_embedding_weight[shape_idx]

        decoder = Decoder(shape_model_log, shape_emb.unsqueeze(0),  device)

        source_cloud = create_mesh(decoder, os.path.join(model_directory, f'Undeformed_{shape_idx}.xyz'), N= 400)
        print(source_cloud.shape)
        # Random Select pointclouds to have the length
        iidx = np.random.permutation(source_cloud.shape[0])[:n]
        source_cloud = source_cloud[iidx,:]
        source_cloud = torch.tensor(source_cloud).unsqueeze(0).float()
        print(source_cloud.size())

        
        target_cloud = data_i['coords'][:,np.where(data_i['gt']==0)[1],:].detach()
        iidx = np.random.permutation(target_cloud.size()[1])[:n]
        target_cloud = target_cloud[:,iidx,:]

        # Chamfer distance
        print(target_cloud.size(), source_cloud.size())
        dist,_ = chamfer_distance(target_cloud, source_cloud, point_reduction = "mean") 

        dist_lst.append(dist.cpu().numpy())
        dist_lst_real.append( 0.5*dist.cpu().numpy() * data_i['scale'] * data_i['scale'])
#         dist_lst_real.append( 0.5*dist.cpu().numpy()/data_i['scale'] )
#         print(dist, 0.5*dist.cpu().numpy()/data_i['scale'] )
        
        

        plt.figure(figsize=(8,8))

        ax = plt.axes(projection='3d')
        ax.view_init(90, -90)
        ax.axis("on")
        plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)

        ax.scatter(source_cloud[:,:,0],source_cloud[:,:,2],source_cloud[:,:,1], s=0.01)
        ax.scatter(target_cloud[:,:,0],target_cloud[:,:,2],target_cloud[:,:,1], s=0.01, c='red')
        ax.set_xlim3d(-1,1)
        ax.set_ylim3d(-1,1)
        ax.set_zlim3d(-1,1)
        ax.grid()
        plt.show()
print(np.mean(dist_lst), np.mean(dist_lst_real))   

In [ ]:
from pytorch3d.loss import chamfer_distance
import matplotlib.pyplot as plt
np.random.seed(0)
random.seed(0)

l1_on = []
l1_off = []
l1_on_real = []
l1_off_real = []


data = data_dict["train"]
for shape_idx in range ( len(data) ): ## YS    
    for deform_idx in data[shape_idx].keys():
        if deform_idx != "nominal":
            continue

        data_i = data[shape_idx][deform_idx]
        shape_emb = shape_embedding_weight[shape_idx]

        decoder = Decoder(shape_model_log, shape_emb.unsqueeze(0),  device)
        pred = decoder(data_i['coords'].to(device))['model_out'][0].detach().cpu()
        
        idx_on = np.where(data_i['gt']==0)[1]
        idx_off = np.where(data_i['gt']!=0)[1]
        
        error =  torch.clip(pred, -0.3, 0.3) - torch.clip(data_i['gt'], -0.3, 0.3)
        error_on = torch.mean( torch.abs(error[:, idx_on, :]))
        error_off = torch.mean(torch.abs(error[:, idx_off, :]))
        
        scale = data_i["scale"]
        print(scale, len(idx_on), len(idx_off))
        l1_on.append( error_on)
        l1_off.append(error_off)
        l1_on_real.append(error_on *scale )
        l1_off_real.append(error_off * scale)
        
print("Result:",np.mean(l1_on), np.mean(l1_off ), np.mean(l1_on_real), np.mean(l1_off_real ) )        
       

## Table 1a : Deformed CD and L1

In [ ]:
from pytorch3d.loss import chamfer_distance
import matplotlib.pyplot as plt
import random
np.random.seed(0)
random.seed(0)

class deform_Decoder(torch.nn.Module):
    def __init__(self,shape_model, deform_model, shape_embedding, deform_embedding, device):
        super().__init__()

        self.shape_embedding = shape_embedding.to(device) #.squeeze()
        self.deform_embedding = deform_embedding.to(device)
        self.shape_model = shape_model.to(device)
        self.deform_model = deform_model.to(device)

    def forward(self, coords, type = 'deformed'):
  
        deform_input={'coords':coords, 
                      'embedding':torch.cat([self.deform_embedding,self.shape_embedding],dim=-1)}
        deform_output = self.deform_model(deform_input)

        shape_input = {'coords': deform_output['model_in'].to(device),
                    'model_out' :deform_output['model_in'].to(device) + deform_output['model_out'],
                    'embedding' : self.shape_embedding}
        
        pred_sdf = self.shape_model(shape_input)
        return pred_sdf
    
    

In [ ]:
dist_lst = []
dist_lst_real = []
n = 5600
data = data_dict["test"] # data_dict["train"]
np.random.seed(0)
random.seed(0)

for shape_idx in range ( len(data) ): ## YS    
    for deform_idx in data[shape_idx].keys():
        if deform_idx == "nominal":
            continue
            
        data_i = data[shape_idx][deform_idx]
        shape_emb = shape_embedding_weight[shape_idx]
        force_emb = deform_embedding_weight[deform_idx]

        decoder = deform_Decoder(shape_model_log, deform_model_log, shape_emb.unsqueeze(0), force_emb, device)

        source_cloud = create_mesh(decoder, os.path.join(model_directory, f'Undeformed_{shape_idx}.xyz'), N= 400)

        # Random Select pointclouds to have the length
        iidx = np.random.permutation(source_cloud.shape[0])[:n]
        source_cloud = source_cloud[iidx,:]
        source_cloud = torch.tensor(source_cloud).unsqueeze(0).float()
        

        
        target_cloud = data_i['coords'][:,np.where(data_i['gt']==0)[1],:].detach()
        iidx = np.random.permutation(target_cloud.size()[1])[:n]
        target_cloud = target_cloud[:,iidx,:]
        print(source_cloud.size(), target_cloud.size())

        dist,_ = chamfer_distance(target_cloud, source_cloud, point_reduction = "mean") 

        dist_lst.append(dist.cpu().numpy())
        dist_lst_real.append( dist.cpu().numpy()* data_i['scale'] * data_i['scale'] )
        print(dist, dist.cpu().numpy()* data_i['scale'] * data_i['scale']  )
        
        

        plt.figure(figsize=(8,8))

        ax = plt.axes(projection='3d')
        ax.view_init(90, -90)
        ax.axis("on")
        plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)

        ax.scatter(source_cloud[:,:,0],source_cloud[:,:,2],source_cloud[:,:,1], s=0.01)
        ax.scatter(target_cloud[:,:,0],target_cloud[:,:,2],target_cloud[:,:,1], s=0.01, c='red')
        ax.set_xlim3d(-1,1)
        ax.set_ylim3d(-1,1)
        ax.set_zlim3d(-1,1)
        ax.grid()
        plt.show()
print(np.mean(dist_lst), np.mean(scaled_dist_lst))

In [ ]:
new_scale = [
0.22752731290415637,
0.22752731290415637,
    0.37394155263900763,
0.29722496451568736,
0.3010651574205548,
0.21852485084636014]

data = data_dict["test"]
for shape_idx in range ( len(data) ): ## YS    
    for deform_idx in data[shape_idx].keys():
        data[shape_idx][deform_idx]["scale"] = new_scale[shape_idx]

In [ ]:
from pytorch3d.loss import chamfer_distance
shape_model.eval()

l1_on = []
l1_off = []
l1_on_real = []
l1_off_real = []

from pytorch3d.loss import chamfer_distance
import matplotlib.pyplot as plt
np.random.seed(0)
random.seed(0)

l1_on = []
l1_off = []
l1_on_real = []
l1_off_real = []


data = data_dict["test"]
for shape_idx in range ( len(data) ): ## YS    
    for deform_idx in data[shape_idx].keys():
        if deform_idx == "nominal":
            continue

        data_i = data[shape_idx][deform_idx]
        shape_emb = shape_embedding_weight[shape_idx]
        force_emb = deform_embedding_weight[deform_idx]

        decoder = deform_Decoder(shape_model_log, deform_model_log, shape_emb.unsqueeze(0), force_emb, device)
        pred = decoder(data_i['coords'].to(device))['model_out'][0].detach().cpu()
        
        idx_on = np.where(data_i['gt']==0)[1]
        idx_off = np.where(data_i['gt']!=0)[1]
        
        error =  torch.clip(pred, -0.3, 0.3) - torch.clip(data_i['gt'], -0.3, 0.3)
        error_on = torch.mean( torch.abs(error[:, idx_on, :]))
        error_off = torch.mean(torch.abs(error[:, idx_off, :]))
        
        scale = data_i["scale"]
        print(scale, len(idx_on), len(idx_off))
        l1_on.append( error_on)
        l1_off.append(error_off)
        l1_on_real.append(error_on *scale )
        l1_off_real.append(error_off * scale)
        
print("Result:",np.mean(l1_on), np.mean(l1_off ), np.mean(l1_on_real), np.mean(l1_off_real ) )     


## Table 1b. Train vs Test

In [ ]:
from pytorch3d.loss import chamfer_distance
import matplotlib.pyplot as plt
np.random.seed(0)
random.seed(0)

dist_lst = []
dist_lst_real = []
n = 5600
data = data_dict["test"]
for shape_idx in range ( len(data) ):    
    for deform_idx in data[shape_idx].keys():
        if deform_idx != "nominal":
            continue
            
        data_i = data[shape_idx][deform_idx]
        shape_emb = shape_embedding_weight[shape_idx]

        decoder = Decoder(shape_model_log, shape_emb.unsqueeze(0),  device)

        source_cloud = create_mesh(decoder, os.path.join(model_directory, f'Undeformed_{shape_idx}.xyz'), N= 400)
        print(source_cloud.shape)
        # Random Select pointclouds to have the length
        iidx = np.random.permutation(source_cloud.shape[0])[:n]
        source_cloud = source_cloud[iidx,:]
        source_cloud = torch.tensor(source_cloud).unsqueeze(0).float()
        print(source_cloud.size())

        
        target_cloud = data_i['coords'][:,np.where(data_i['gt']==0)[1],:].detach()
        iidx = np.random.permutation(target_cloud.size()[1])[:n]
        target_cloud = target_cloud[:,iidx,:]

        # Chamfer distance
        print(target_cloud.size(), source_cloud.size())
        dist,_ = chamfer_distance(target_cloud, source_cloud, point_reduction = "mean") 

        dist_lst.append(dist.cpu().numpy())
        dist_lst_real.append( 0.5*dist.cpu().numpy() * data_i['scale'] * data_i['scale'])
#         dist_lst_real.append( 0.5*dist.cpu().numpy()/data_i['scale'] )
#         print(dist, 0.5*dist.cpu().numpy()/data_i['scale'] )
        
        

        plt.figure(figsize=(8,8))

        ax = plt.axes(projection='3d')
        ax.view_init(90, -90)
        ax.axis("on")
        plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)

        ax.scatter(source_cloud[:,:,0],source_cloud[:,:,2],source_cloud[:,:,1], s=0.01)
        ax.scatter(target_cloud[:,:,0],target_cloud[:,:,2],target_cloud[:,:,1], s=0.01, c='red')
        ax.set_xlim3d(-1,1)
        ax.set_ylim3d(-1,1)
        ax.set_zlim3d(-1,1)
        ax.grid()
        plt.show()
print(np.mean(dist_lst), np.mean(dist_lst_real))   